# LAB | Imbalanced

**Load the data**

In this challenge, we will be working with Credit Card Fraud dataset.

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv

Metadata

- **distance_from_home:** the distance from home where the transaction happened.
- **distance_from_last_transaction:** the distance from last transaction happened.
- **ratio_to_median_purchase_price:** Ratio of purchased price transaction to median purchase price.
- **repeat_retailer:** Is the transaction happened from same retailer.
- **used_chip:** Is the transaction through chip (credit card).
- **used_pin_number:** Is the transaction happened by using PIN number.
- **online_order:** Is the transaction an online order.
- **fraud:** Is the transaction fraudulent. **0=legit** -  **1=fraud**


In [ ]:
#Libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
    precision_recall_fscore_support
)

# Imbalanced learning tools
# If you don't have it installed: pip install imbalanced-learn
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [4]:
fraud = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv")
fraud.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


**Steps:**

- **1.** What is the distribution of our target variable? Can we say we're dealing with an imbalanced dataset?
- **2.** Train a LogisticRegression.
- **3.** Evaluate your model. Take in consideration class importance, and evaluate it by selection the correct metric.
- **4.** Run **Oversample** in order to balance our target variable and repeat the steps above, now with balanced data. Does it improve the performance of our model? 
- **5.** Now, run **Undersample** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model?
- **6.** Finally, run **SMOTE** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model? 

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
    precision_recall_fscore_support
)

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler


url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv"
df = pd.read_csv(url)

TARGET = "fraud"

assert TARGET in df.columns, f"Target column '{TARGET}' not found."

X = df.drop(columns=[TARGET])
y = df[TARGET].astype(int)


counts = y.value_counts().sort_index()
pct = y.value_counts(normalize=True).sort_index() * 100

print("Target distribution (0=legit, 1=fraud):")
print(pd.DataFrame({"count": counts, "percent": pct.round(3)}))
print()

minority_pct = pct.loc[1] if 1 in pct.index else 0
if minority_pct < 20:
    print(f"Imbalanced dataset detected: fraud class is ~{minority_pct:.3f}% of data.\n")
else:
    print("Dataset does not appear strongly imbalanced.\n")


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


def evaluate_model(name, model, X_test, y_test):
    """
    Prints metrics suitable for imbalanced classification.
    Focus: minority (fraud=1) precision/recall/F1 + PR-AUC.
    """
    y_pred = model.predict(X_test)

    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_proba = None


    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average=None, labels=[0, 1]
    )
    p1, r1, f1_1 = precision[1], recall[1], f1[1]

    print(f"=== {name} ===")
    print("Confusion matrix [ [TN FP], [FN TP] ]:")
    print(confusion_matrix(y_test, y_pred))

    print(f"\nFraud class (1) metrics:")
    print(f"Precision: {p1:.4f}")
    print(f"Recall   : {r1:.4f}")
    print(f"F1-score : {f1_1:.4f}")

    
    if y_proba is not None:
        roc_auc = roc_auc_score(y_test, y_proba)
        pr_auc = average_precision_score(y_test, y_proba)  # PR-AUC (Average Precision)
        print(f"\nROC-AUC  : {roc_auc:.4f}")
        print(f"PR-AUC   : {pr_auc:.4f}   (recommended for imbalanced data)")
    else:
        print("\nROC-AUC / PR-AUC not available (no probabilities).")

    print("\nClassification report:")
    print(classification_report(y_test, y_pred, digits=4))
    print("-" * 60)
    print()

    
    return {
        "model": name,
        "precision_fraud": p1,
        "recall_fraud": r1,
        "f1_fraud": f1_1,
        "roc_auc": roc_auc if y_proba is not None else np.nan,
        "pr_auc": pr_auc if y_proba is not None else np.nan
    }



clf = LogisticRegression(max_iter=2000, solver="lbfgs")  # baseline LR


baseline_pipeline = ImbPipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", clf)
])

baseline_pipeline.fit(X_train, y_train)
baseline_results = evaluate_model("Baseline LogisticRegression", baseline_pipeline, X_test, y_test)


oversample_pipeline = ImbPipeline(steps=[
    ("scaler", StandardScaler()),
    ("sampler", RandomOverSampler(random_state=42)),
    ("model", LogisticRegression(max_iter=2000, solver="lbfgs"))
])

oversample_pipeline.fit(X_train, y_train)
over_results = evaluate_model("RandomOverSampler + LogisticRegression", oversample_pipeline, X_test, y_test)


undersample_pipeline = ImbPipeline(steps=[
    ("scaler", StandardScaler()),
    ("sampler", RandomUnderSampler(random_state=42)),
    ("model", LogisticRegression(max_iter=2000, solver="lbfgs"))
])

undersample_pipeline.fit(X_train, y_train)
under_results = evaluate_model("RandomUnderSampler + LogisticRegression", undersample_pipeline, X_test, y_test)


smote_pipeline = ImbPipeline(steps=[
    ("scaler", StandardScaler()),
    ("sampler", SMOTE(random_state=42, k_neighbors=5)),
    ("model", LogisticRegression(max_iter=2000, solver="lbfgs"))
])

smote_pipeline.fit(X_train, y_train)
smote_results = evaluate_model("SMOTE + LogisticRegression", smote_pipeline, X_test, y_test)


summary = pd.DataFrame([baseline_results, over_results, under_results, smote_results])
summary = summary.sort_values(by="pr_auc", ascending=False)

print("==== Summary (sorted by PR-AUC) ====")
print(summary[["model", "precision_fraud", "recall_fraud", "f1_fraud", "roc_auc", "pr_auc"]].round(4))
print()

print("Interpretation guidance:")
print("- For imbalanced problems, PR-AUC and Recall/F1 on the fraud class are often more informative than accuracy.")
print("- Oversampling/SMOTE typically increases recall (catches more fraud) but may reduce precision (more false alarms).")
print("- Undersampling can simplify learning but may lose important majority-class information.")


Target distribution (0=legit, 1=fraud):
        count  percent
fraud                 
0      912597    91.26
1       87403     8.74

Imbalanced dataset detected: fraud class is ~8.740% of data.

=== Baseline LogisticRegression ===
Confusion matrix [ [TN FP], [FN TP] ]:
[[181296   1223]
 [  6895  10586]]

Fraud class (1) metrics:
Precision: 0.8964
Recall   : 0.6056
F1-score : 0.7228

ROC-AUC  : 0.9670
PR-AUC   : 0.8072   (recommended for imbalanced data)

Classification report:
              precision    recall  f1-score   support

           0     0.9634    0.9933    0.9781    182519
           1     0.8964    0.6056    0.7228     17481

    accuracy                         0.9594    200000
   macro avg     0.9299    0.7994    0.8505    200000
weighted avg     0.9575    0.9594    0.9558    200000

------------------------------------------------------------

=== RandomOverSampler + LogisticRegression ===
Confusion matrix [ [TN FP], [FN TP] ]:
[[170390  12129]
 [   911  16570]]

Fraud c

In [ ]:
"""
Conclusion:

The dataset is highly imbalanced, with only 8.74% fraudulent transactions.

The baseline Logistic Regression model achieved high precision (0.896) but lower recall (0.606),
meaning many fraud cases were missed.

After applying Oversampling, Undersampling, and SMOTE, recall increased significantly (~0.95),
allowing the model to detect most fraud cases, but precision decreased (~0.58), resulting in more false positives.

PR-AUC was highest for the baseline model (0.807), while resampling methods achieved slightly lower PR-AUC (~0.757).

This shows a trade-off:
- Baseline model is better when minimizing false alarms.
- Resampling models are better when maximizing fraud detection.

Depending on business priorities, SMOTE or oversampling may be preferred if catching fraud is more important than precision.
"""
